# `mkdocs` plugin for jupyter notebooks

i think i want more control of how `mkdocs` renders notebooks.
i've been using [`mkdocs-jupyter`][mkdocs-jupyter] for a while and it is great,
but i need more knobs.

my particular need is to configure `nbconvert` exporters with more fine grain control than what [`mkdocs-jupyter`][mkdocs-jupyter] offers.
the major difference is we are going to target markdown output rather than html.

[mkdocs-jupyter]: https://github.com/danielfrg/mkdocs-jupyter
[plugin]: https://www.mkdocs.org/dev-guide/plugins/

## checklist for successfully integrating the plugin

steps to adding a [`mkdocs` plugin][plugin] to this the `tonyfast` project:

- [x] add plugin to `mkdocs.yml`

  ```yaml
  plugins:
      - markdown_notebook
  ```
  
- [x] define plugin entry point for `tonyfast`

   ```toml
   [project.entry-points."mkdocs.plugins"]
   markdown_notebook = "tonyfast.mkdocs:MarkdownNotebook"
   ```

- [x] build the `MarkdownNotebook` plugin 
- [x] integrate the plugin

   ```python
   from tonyfast.mkdocs import MarkdownNotebook
   ```
   
- [ ] add and improve the `nbconvert` export display renderers

[mkdocs-jupyter]: https://github.com/danielfrg/mkdocs-jupyter
[plugin]: https://www.mkdocs.org/dev-guide/plugins/

## building the [`mkdocs` plugin][plugin]

[mkdocs-jupyter]: https://github.com/danielfrg/mkdocs-jupyter
[plugin]: https://www.mkdocs.org/dev-guide/plugins/

In [ ]:
    %pip install jinja2 mkdocs tonyfast markdown nbconvert nbformat
    import json, nbconvert, nbformat, pathlib, mkdocs.plugins, warnings, re, functools
    warnings.filterwarnings("ignore", category=DeprecationWarning)

the [`mkdocs` plugin]

we now use a custom template and expo defined in 2022-12-31-markdownish-notebook.ipynb

In [ ]:
        with __import__("importnb").Notebook():
            from tonyfast.xxii.__markdownish_notebook import template, PidgyExporter

In [ ]:
    class MarkdownNotebook(mkdocs.plugins.BasePlugin):
        exporter_cls = PidgyExporter
        config_scheme = (
            # ('foo', mkdocs.config.config_options.Type(str, default='a default value')),
        )
                    
        @functools.lru_cache
        def get_exporter(self, key="mkdocs", **kw):
            with __import__("importnb").Notebook():
                from tonyfast.xxii.__markdownish_notebook import template, HEAD, replace_attachments
            kw.setdefault("template_file", key)
            exporter = self.exporter_cls(**kw)
            exporter.environment.filters.setdefault("attachment", replace_attachments)
            from jinja2 import DictLoader
            for loader in exporter.environment.loader.loaders:
                if isinstance(loader, DictLoader):
                    loader.mapping[key] = template
                    loader.mapping["HEAD"] = HEAD
                    break
            return exporter
            
        def on_page_read_source(self, page, config):
            if page.file.is_modified():
                if page.file.src_uri.endswith((".ipynb", )):
                    body = pathlib.Path(page.file.abs_src_path).read_text()
                    nb = nbformat.v4.reads(body)
                    exporter = self.get_exporter()
                    return "\n".join((
                        "---", json.dumps(nb.metadata),"---", # add metadata as front matter
                        exporter.from_notebook_node(nb)[0]
                    ))
                
        def on_post_page(self, output, page, config):
            if '<script type="application/vnd.jupyter.widget-view+json">' in output:
                left, sep, right = output.partition("</head")
                exporter = self.get_exporter()
                return left + self.get_exporter().environment.get_template("HEAD").render(resources=dict(
                            jupyter_widgets_base_url=exporter.jupyter_widgets_base_url, 
                            html_manager_semver_range=exporter.html_manager_semver_range, 
                            widget_renderer_url=exporter.widget_renderer_url,
                            require_js_url=exporter.require_js_url
                        )) + sep + right
            
        def on_page_markdown(self, markdown, page, config, files):
            import markdown
            title = markdown.Markdown(extensions=config['markdown_extensions']).convert(page.title)
            page.title = title[len("<p>"):-len("</p>")].strip().replace("code>", "pre>")

we trick `mkdocs` into thinking notebook files are markdown extensions.

In [ ]:
    mkdocs.utils.markdown_extensions += ".ipynb", # this feels naughty.